# Tutorial: Creating Custom Aggregates with DynamicEventAggregator
This tutorial will guide you through using the `DynamicEventAggregator` to generate standard and custom event aggregates from match data.

## 📋 Prerequisites
- Python installed
- Access to the `skillcorner_analysis_lib` library
- Required libraries: `numpy`, `pandas`
```bash
pip install numpy pandas skillcorner
```

In [1]:
import numpy as np
import pandas as pd
from io import BytesIO
from DynamicEventsAggregator import DynamicEventAggregator
from skillcorner.client import SkillcornerClient

/Users/nano/PycharmProjects/opendata/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## ⚽ Step 2: Load Match Data
We will use the `GCSHelper` to load dynamic event data for a specific match.

In [7]:
match_id = 1886347

# Instantiate the SkillCorner client
username = "your_username"
password = "your_password"
client = SkillcornerClient(username=username, password=password)

# Load dynamic event data from SkillCorner API
# events_df = pd.read_csv(BytesIO(client.get_dynamic_events(MATCH_ID)))

# Load dynamic event data from Open Source
events_df = pd.read_csv(
    f"../../../data/matches/{match_id}/{match_id}_dynamic_events.csv"
)

## 📊 Step 3: Generate Standard Aggregates
Create an instance of `DynamicEventAggregator` and generate predefined aggregates like off-ball runs, line-breaking passes, and defensive engagements.

In [8]:
# Initialize the aggregator
events_aggregator = DynamicEventAggregator(df=events_df)

# Off-ball runs
off_ball_runs = events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="off_ball_runs"
)

# Line-breaking passes
line_breaking_passes = events_aggregator.generate_aggregates(
    group_by=["player_in_possession_id", "player_in_possession_name"],
    aggregate_type="line_breaking_passes",
)

# Defensive engagements
defensive_engagements = events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="on_ball_engagements"
)

# Pressing
pressing = events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="pressing_engagements"
)

off_ball_runs.head()

/Users/nano/PycharmProjects/opendata/resources/Tutorials/Aggregating Dynamic Events/DynamicEventsAggregator.py:948: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  context_df.groupby(group_by + ['context'])
/Users/nano/PycharmProjects/opendata/resources/Tutorials/Aggregating Dynamic Events/DynamicEventsAggregator.py:948: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  context_df.groupby(group_by + ['context'])
/Users/nan

,player_id,player_name,count_off_ball_runs,count_targeted_off_ball_runs,count_received_off_ball_runs,xthreat_off_ball_runs,xthreat_targeted_off_ball_runs,xthreat_received_off_ball_runs,xpass_completion_off_ball_runs,xpass_completion_targeted_off_ball_runs,...,count_dangerous_received_dropping_off_runs_in_create,count_difficult_dropping_off_runs_in_create,count_difficult_targeted_dropping_off_runs_in_create,count_difficult_received_dropping_off_runs_in_create,avg_speed_avg_dropping_off_runs_in_create,count_hsr_dropping_off_runs_in_create,count_sprint_dropping_off_runs_in_create,avg_distance_covered_dropping_off_runs_in_create,count_center_channel_dropping_off_runs_in_create,count_wide_channel_dropping_off_runs_in_create
0,14736,L. Verstraete,29.0,9.0,9.0,0.1767,0.0312,0.0312,25.5104,8.7506,...,0.0,0.0,0.0,0.0,17.2325,0.0,0.0,17.4000,1.0,0.0
1,23418,F. Gallegos,48.0,18.0,16.0,0.5688,0.2346,0.1967,39.9419,14.8335,...,0.0,0.0,0.0,0.0,15.8775,0.0,0.0,6.2575,0.0,1.0
2,33697,N. Pijnaker,13.0,4.0,4.0,0.0168,0.0006,0.0006,12.4500,3.9792,...,0.0,0.0,0.0,0.0,16.6400,0.0,0.0,9.3500,3.0,1.0
3,38673,G. May,44.0,16.0,8.0,1.7826,0.6253,0.3767,25.6969,8.7082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43829,N. Moreno,13.0,3.0,2.0,0.3192,0.0215,0.0065,8.4089,2.1079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Note there are now 821 columns !

## 🛠️ Step 4: Create Custom Aggregates
You can define your own contexts and metrics to tailor the analysis.

### Define Custom Contexts

In [9]:
contexts = {
    "custom": {
        "all_possessions": (
            (events_df["event_type"] == "player_possession")
            & (events_df["team_in_possession_phase_type"] == "finish")
            & (events_df["separation_start"] >= 5)
        )
    }
}

### Define Custom Metrics

In [10]:
metric = {
    "custom": {
        "count": lambda x: len(x),
        "avg_duration": lambda x: x["duration"].mean(),
        "avg_distance_covered": lambda x: x["distance_covered"].mean(),
    }
}

### Generate Custom Aggregates

In [11]:
# Initialize aggregator with custom contexts and metrics
custom_events_aggregator = DynamicEventAggregator(
    df=events_df, custom_context_groups=contexts, custom_metric_groups=metric
)

# Generate custom aggregates
custom_aggregates = custom_events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="custom"
)

# Display the results
custom_aggregates.head()

/Users/nano/PycharmProjects/opendata/resources/Tutorials/Aggregating Dynamic Events/DynamicEventsAggregator.py:948: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  context_df.groupby(group_by + ['context'])


,player_id,player_name,count_all_possessions,avg_duration_all_possessions,avg_distance_covered_all_possessions
0,14736,L. Verstraete,9.0,1.966667,4.686667
1,23418,F. Gallegos,10.0,1.020000,2.167000
2,33697,N. Pijnaker,12.0,1.066667,3.006667
3,38673,G. May,2.0,2.000000,10.705000
4,43829,N. Moreno,2.0,2.800000,10.400000
